# 🔥🌐 Gemini Prompt Freeform - TRANSLATOR 🌐🔥

[GitHub](https://github.com/MattyMroz/GeminiTranslator)

[Gemini Playground](https://makersuite.google.com/app/prompts)

[Python Quickstart Gemini](https://ai.google.dev/tutorials/python_quickstart)



# 🔥🌐 GOOGLE COLAB GEMINI TRANSLATOR 🌐🔥
## ✨ TŁUMACZENIE NAPISÓW I KSIĄŻEK - PRZYJMOWANY FORMAT .SRT ✨

### ⚡INSTRUKCJA:
1. Otwórz swój notatnik w Google Colab.
2. Wybierz lokalizacjie docelową ustawiając os.chdir('twoja/lokalizacja')
```python
    os.chdir('/content/drive/Shareddrives/Dysk_0/')
    os.makedirs('GeminiTranslator', exist_ok=True)

    os.chdir('/content/drive/Shareddrives/Dysk_0/GeminiTranslator/')
    os.makedirs('input', exist_ok=True)
    os.makedirs('output', exist_ok=True)
```
3. Możesz zmienić prompt docelowy w kodzie.
4. Prześlij pliki .srt do przetłumaczenia.
5. Ustaw swoje ustawienia lub dodaj dodatkowe informacjie o tłumaczonym tekście np.: płeć postaci, opis fabuły, inne istotnie informacjie w kontekście tłumaczenia.
6. Zacznij tłumaczyć

<details>
<summary>5. Kod JavaScript do Skopiowania ✨</summary>

```
TESTR
```
</details>







# 1. Gemini Translator - Tłumacz Napisów SRT


In [ ]:
# @title 1.1 Konfiguracja Wstępna { run: "auto" }
!pip install googletrans==3.1.0a0 -U
!pip install pysrt -U
!pip install google-generativeai -U -q
# !pip install natsort -U
# !pip install rich -U

In [ ]:
# @title 1.2 Gemini Translator - Tłumacz Napisów SRT 1.0 { run: "auto" }

from contextlib import suppress
from dataclasses import dataclass
from datetime import datetime
from time import perf_counter_ns
from typing import List, Optional

import google.generativeai as genai
import ipywidgets as widgets
import os
import pysrt
import re
import shutil
import zipfile

from google.colab import files
from googletrans import Translator
from IPython.display import display, HTML
from natsort import natsorted
from rich.console import Console
from rich.theme import Theme

# ===============================================================================
# Button CSS
display(HTML('''
<style>
    .widget-button {
        font-weight: bold !important;
    }
    .widget-button.jupyter-widgets.widget-button.mod-active,
    .widget-button:active {
        background-color: #2d2e2e;
    }
    .widget-toggle-button {
        font-weight: bold !important;
    }
    .widget-toggle-button.jupyter-widgets.widget-toggle-button.mod-active,
    .widget-toggle-button:active {
        background-color: #2d2e2e;
    }
</style>
'''))
# ===============================================================================

# ===============================================================================
# Rich print styles
console: Console = Console(theme=Theme({
    "purple_bold": "purple bold",
    "purple_italic": "purple italic",
    "pink_bold": "pale_violet_red1 bold",
    "pink_italic": "pale_violet_red1 italic",
    "red_bold": "bright_red bold",
    "red_italic": "bright_red italic",
    "brown_bold": "rgb(180,82,45) bold",
    "brown_italic": "rgb(180,82,45) italic",
    "orange_bold": "rgb(255,135,70) bold",
    "orange_italic": "rgb(255,135,70) italic",
    "yellow_bold": "bright_yellow bold",
    "yellow_italic": "bright_yellow italic",
    "green_bold": "green bold",
    "green_italic": "green italic",
    "blue_bold": "dodger_blue2 bold",
    "blue_italic": "dodger_blue2 italic",
    "white_bold": "white bold",
    "white_italic": "white italic",
    "normal_bold": "bold",
    "normal_italic": "italic",
    "black_bold": "rgb(0,0,0) on white bold",
    "black_italic": "rgb(0,0,0) on white italic",
    "repr.number": "bright_red bold",
}))
# ===============================================================================

# ===============================================================================
# ExecutionTimer - Mierzenie Czasu
"""
    Module execution_timer provides an ExecutionTimer class
        to measure the execution time of a code block.
    It offers two usage options: as a context manager and as a decorator.

    * Example usage as a context manager:
        with ExecutionTimer():
            main()

    * Example usage as a decorator:
        @execution_timer
        def main():
            # Code block to measure execution time
"""


@dataclass(slots=True)
class ExecutionTimer:
    """
        ExecutionTimer is a context manager that measures the execution time of a code block.
        It captures the start time, end time, and duration of the code block.
    """

    start_date: datetime = None
    end_date: datetime = None
    start_time_ns: int = None
    end_time_ns: int = None
    console: Console = Console()

    def __post_init__(self):
        self.start_date = datetime.now()
        self.start_time_ns = perf_counter_ns()

    def __enter__(self) -> 'ExecutionTimer':
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        try:
            self.end_date = datetime.now()
            self.end_time_ns = perf_counter_ns()
            self.display_time()
        except AttributeError:
            print('An error occurred: __exit__')

    @staticmethod
    def current_datetime(date: datetime) -> str:
        """
            Formats a datetime object as a string in the format 'YYYY-MM-DD HH:MM:SS'.
        """

        return f'[yellow]{date.year}-{date.month:02d}-{date.day:02d}' \
               f' [white bold]{date.hour:02d}:{date.minute:02d}:{date.second:02d}'

    def calculate_duration(self) -> str:
        """
            Calculates the duration of the code block in hours, minutes, seconds, milliseconds,
            microseconds, and nanoseconds.
        """

        duration_ns: int = self.end_time_ns - self.start_time_ns
        duration_s, duration_ns = map(int, divmod(duration_ns, 1_000_000_000))
        duration_ms, duration_ns = map(int, divmod(duration_ns, 1_000_000))
        duration_us, duration_ns = map(int, divmod(duration_ns, 1_000))

        hours, remainder = map(int, divmod(duration_s, 3600))
        minutes, seconds = map(int, divmod(remainder, 60))

        return f'[white bold]{hours:02d}:{minutes:02d}:{seconds:02d}:' \
               f'{duration_ms:03d}:{duration_us:03d}:{duration_ns:03d}'

    def calculate_duration_alt(self) -> tuple[float, ...]:
        """
            Calculates the duration of the code block in hours, minutes, and seconds
            using an alternative method.
        """

        duration_ns: int = self.end_time_ns - self.start_time_ns
        hours_alt: float = duration_ns / 1_000_000_000 / 60 / 60
        minutes_alt: float = duration_ns / 1_000_000_000 / 60
        seconds_alt: float = duration_ns / 1_000_000_000

        return hours_alt, minutes_alt, seconds_alt

    def display_time(self):
        """
            Displays the start date, end date, and duration of the code block execution.
        """

        start_date_str: str = self.current_datetime(self.start_date)
        end_date_str: str = self.current_datetime(self.end_date)
        duration: str = self.calculate_duration()
        hours_alt, minutes_alt, seconds_alt = map(
            float, self.calculate_duration_alt())

        self.console.print(
            '\n[bold white]╚═══════════ EXECUTION TIME ═══════════╝')
        self.console.print(
            '[bold bright_yellow]        YYYY-MM-DD HH:MM:SS:ms :µs :ns')
        self.console.print(
            f'[bright_red bold][[bold white]START[bright_red bold]] {start_date_str}')
        self.console.print(
            f'[bright_red bold][[bold white]END[bright_red bold]]   {end_date_str}')
        self.console.print(
            f'[bright_red bold][[bold white]TIME[bright_red bold]]  [bold bright_yellow]YYYY-MM-DD {duration}')
        self.console.print('[bright_red bold]                   ^^^^^^^^^^^^')
        self.console.print(
            f'[bright_red bold][[bold white]TIME[bright_red bold]]  [white bold]{hours_alt:.9f} hours')
        self.console.print(
            f'[bright_red bold][[bold white]TIME[bright_red bold]]  [white bold]{minutes_alt:.9f} minutes')
        self.console.print(
            f'[bright_red bold][[bold white]TIME[bright_red bold]]  [white bold]{seconds_alt:.9f} seconds')


def execution_timer(func):
    """
        Decorator that measures the execution time of a function using ExecutionTimer.
    """

    def wrapper(*args, **kwargs):
        with ExecutionTimer():
            result = func(*args, **kwargs)
        return result

    return wrapper
# ===============================================================================

# ===============================================================================


class GeminiTranslator:
    """
        The GeminiTranslator class provides methods for translating subtitles using Gemini.

        Attributes:
            - input_folder (str): The path to the input folder.
            - output_folder (str): The path to the output folder.
            - api_key (str): The API key for Gemini.
            - translator (Translator): The Google Translator object.
            - auto_download (bool): Whether to automatically download the output files.

        Methods:

    """

    def __init__(self, input_folder: str, output_folder: str, api_key: str):
        """
            Initializes the GeminiTranslator instance.

            Attributes:
                - input_folder (str): The path to the folder where the input files are located.
                - output_folder (str): The path to the folder where the output files will be saved.
                - api_key (str): The API key for Gemini.

            The method also sets up various widgets for user interaction, including sliders, dropdowns, and buttons. These widgets allow the user to adjust translation parameters, clear input/output folders, upload .SRT files, toggle automatic downloading, start the translation process, and download the resulting files.
        """
        button_layout = widgets.Layout(width='300px', height='30px')
        button_style = dict(font_weight='bold')

        self.input_folder: str = input_folder
        self.output_folder: str = output_folder
        self.api_key: str = api_key
        self.translator: Translator = Translator()
        self.auto_download: bool = False

        self.api_key = widgets.Text(
            value=api_key, description='API_KEY', continuous_update=False)
        self.temperature = widgets.FloatSlider(
            value=0.40, min=0, max=1, step=0.05, description='Temperature')
        self.top_p = widgets.IntSlider(
            value=1, min=0, max=40, step=1, description='Top P')
        self.top_k = widgets.IntSlider(
            value=1, min=0, max=40, step=1, description='Top K')
        self.max_output_tokens = widgets.IntSlider(
            value=1000000, min=1, max=1000000, step=1, description='Max Output Tokens')
        self.translated_line_count = widgets.SelectionSlider(value=50, options=[
                                                             1, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], description='Lines to Translate')
        self.safety_harassment = widgets.Dropdown(
            options=['BLOCK_ONLY_HIGH', 'BLOCK_MEDIUM_AND_ABOVE', 'BLOCK_LOW_AND_ABOVE'], description='Harassment')
        self.safety_hate_speech = widgets.Dropdown(
            options=['BLOCK_ONLY_HIGH', 'BLOCK_MEDIUM_AND_ABOVE', 'BLOCK_LOW_AND_ABOVE'], description='Hate Speech')
        self.safety_sexually_explicit = widgets.Dropdown(
            options=['BLOCK_ONLY_HIGH', 'BLOCK_MEDIUM_AND_ABOVE', 'BLOCK_LOW_AND_ABOVE'], description='Sexually Explicit')
        self.safety_dangerous_content = widgets.Dropdown(
            options=['BLOCK_ONLY_HIGH', 'BLOCK_MEDIUM_AND_ABOVE', 'BLOCK_LOW_AND_ABOVE'], description='Dangerous Content')
        self.translation_notes = widgets.Textarea(
            value='', description='Uwagi Tłumaczeniowe', continuous_update=False, layout=widgets.Layout(width="90%", height="60px"))
        self.translation_mode = widgets.Dropdown(
            options=['Gemini', 'Gemini + Google Translator'], description='Tryb:')

        self.clear_input_button = widgets.Button(
            description='Usuń Pliki z input', style=button_style, layout=button_layout)
        self.clear_input_button.on_click(self.clear_input_folder)
        self.clear_output_button = widgets.Button(
            description='Usuń Pliki z output', style=button_style, layout=button_layout)
        self.clear_output_button.on_click(self.clear_output_folder)

        self.upload_button = widgets.Button(
            description='Prześlij Plik .SRT', multiple=True, style=button_style, layout=button_layout)
        self.upload_button.on_click(self.upload_srt_files)

        self.auto_download_button = widgets.ToggleButton(
            value=False, description='Automatyczne Pobieranie', style=button_style, layout=button_layout)
        self.auto_download_button.observe(
            self.update_auto_download_status, names='value')

        self.start_translate_button = widgets.Button(
            description='Tłumacz Napisy', style=button_style, layout=button_layout)
        self.start_translate_button.on_click(self.start_translation)

        self.download_button = widgets.Button(
            description='Pobierz', style=button_style, layout=button_layout)
        self.download_button.on_click(self.download_files)

    # Metody wykonawcze
    def translate_google(self, filename: str, translated_line_count: int, is_combined_with_gemini: bool = False) -> pysrt.SubRipFile:
        """
            Translates subtitles using Google Translate.

            Attributes:
                - filename (str): The name of the subtitle file to be translated.
                - translated_line_count (int): The number of subtitle lines to be translated at a time.
                - is_combined_with_gemini (bool, optional): Flag indicating whether the translation should be combined with Gemini. Defaults to False.

            Returns:
                - pysrt.SubRipFile: The translated subtitle file.

            This method opens the subtitle file, iterates over the subtitles, and combines them into groups based on the translated_line_count. Each group of lines is then translated using Google Translate. The translated text is split back into individual lines and assigned back to the original subtitles. If is_combined_with_gemini is True, the translated subtitles are saved as a temporary file and returned. Otherwise, the translated subtitles are saved directly to the output folder.
        """
        subs: pysrt.SubRipFile = pysrt.open(
            os.path.join(self.input_folder, filename), encoding='utf-8')
        subs_combined: List[str] = []
        translated_subs: List[str] = []

        translator: Translator = Translator()
        for i, sub in enumerate(subs):
            sub.text = sub.text.replace("\n", " ◍ ")
            subs_combined.append(sub.text)

            if (i + 1) % translated_line_count == 0 or i == len(subs) - 1:
                combined_text: str = "\n".join(subs_combined)
                translated_text: str = translator.translate(
                    combined_text, dest='pl').text
                translated_subs += translated_text.split("\n")
                subs_combined = []

        for i, sub in enumerate(subs):
            sub.text = translated_subs[i]
            sub.text = sub.text.replace(" ◍, ", ",\n")
            sub.text = sub.text.replace(" ◍ ", "\n")
            sub.text = sub.text.replace(" ◍", "")

        if is_combined_with_gemini:
            translated_filename: str = filename.replace(
                '.srt', '_translated_temp.srt')
            subs.save(os.path.join(self.input_folder, translated_filename))
            return subs
        subs.save(os.path.join(self.output_folder, filename))

    def translate_google_gemini(self, filename: str, translated_line_count: int) -> None:
        """
            Translates subtitles using both Google Translate and Gemini.

            Attributes:
                - filename (str): The name of the subtitle file to be translated.
                - translated_line_count (int): The number of subtitle lines to be translated at a time.

            This method first translates the subtitles using Google Translate and then refines the translation using Gemini. The translated subtitles are saved as a temporary file, which is then removed after the Gemini translation is complete.
        """
        translated_subs: pysrt.SubRipFile = self.translate_google(
            filename, translated_line_count, is_combined_with_gemini=True)
        self.translate_gemini(
            filename, translated_line_count, translated_subs)
        os.remove(os.path.join(self.input_folder, filename.replace(
            '.srt', '_translated_temp.srt')))

    def translate_gemini(self, filename: str, translated_line_count: int, translated_subs: Optional[pysrt.SubRipFile] = None):
        """
            Translates subtitles using Gemini.

            Attributes:
                - filename (str): The name of the subtitle file.
                - translated_line_count (int): The number of lines to translate at a time.
                - translated_subs (Optional[pysrt.SubRipFile], optional): The translated subtitles. Defaults to None.

            This method configures the Gemini API with the provided API key, sets up the generation and safety settings, and opens the subtitle file. It then divides the subtitles into groups based on the translated_line_count and translates each group using Gemini. If translated_subs is provided, it is used as a preliminary translation. The translated subtitles are saved to the output folder.
        """
        genai.configure(api_key=self.api_key.value)

        generation_config: dict = {
            "temperature": self.temperature.value,
            "top_p": self.top_p.value,
            "top_k": self.top_k.value,
            "max_output_tokens": self.max_output_tokens.value,
        }

        safety_settings: dict = [
            {
                "category": "HARM_CATEGORY_HARASSMENT",
                "threshold": self.safety_harassment.value
            },
            {
                "category": "HARM_CATEGORY_HATE_SPEECH",
                "threshold": self.safety_hate_speech.value
            },
            {
                "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
                "threshold": self.safety_sexually_explicit.value
            },
            {
                "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
                "threshold": self.safety_dangerous_content.value
            },
        ]

        model = genai.GenerativeModel(model_name="gemini-pro",
                                      generation_config=generation_config,
                                      safety_settings=safety_settings)

        subs: pysrt.SubRipFile = pysrt.open(
            os.path.join(self.input_folder, filename), encoding='utf-8')
        groups: List[List[pysrt.SubRipItem]] = [subs[i:i+translated_line_count]
                                                for i in range(0, len(subs), translated_line_count)]

        counter: int = 1
        for group in groups:
            text: str = ""
            for sub in group:
                text += "◍◍{}. {}".format(counter,
                                          sub.text.replace('\n', ' ◍◍◍◍ ')) + " @@\n"
                counter += 1
            text = text.rstrip(' @@\n')

            prompt = """
        Jesteś moim profesjonalnym tłumaczem i polonistą z nieskończonym doświadczeniem w tłumaczeniu i poprawianiu wszystkich błędów w wszelkiego rodzaju tekstach. Twoje zadania to:
        1. Identyfikowanie języka źródłowego
        2. Tłumaczenie tekstu na język polski
        3. Poprawianie i ulepszanie tłumaczenia, tak aby był pozbawiony błędów i był jak najbardziej wiernie odwzorowany na oryginał

        BEZWZGLĘDNE ZASADY KTÓRYCH NIE PRZESTRZEGANIE BĘDZIE SKUTKOWAŁO ODRZUCENIEM TŁUMACZENIA:

        1. FORMATOWANIE:
            - JEŚLI NA POCZĄTKU PROMPTU JEST NAPISANE ŻE POPEŁNIŁEŚ BŁĄD TO SPRAW, ŻE TO JUŻ SIĘ NIE POWTÓRZY ZA TYM RAZEM
            - Zachowaj oryginalne formatowanie tekstu.
            - Zachowaj znaki specjalne i nie zmienaj ich ilości: '◍◍◍◍', '@@' lub '◍◍[num]'
            - Jak otrzymasz 30 napisów, zwróć 30 napisów.
            - Nie łącz, nie sumuj nie zmianiaj układu i struktury napisów
            - Nie usuwaj, nie dodawaj, nie zmieniaj znaków interpunkcyjnych
            - Nie wyjaśnaj, nie komentuj, zwróć polski lepszy klon oryginału

        2. TŁUMACZENIE:
            - Podejdź globalnie do tekstu, uwzględniając kontekst i informacje o płci postaci
            - Bądź kreatywny, dostosowując swój ton do kontekstu
            - Używaj wulgaryzmów i nie łagoć ich: kurwa, chuj, pierdolić, jebać, cholera
            - Zwróć uwagę na poprawność gramatyczną, składnię i interpunkcję
            - Bądź świadomy różnic między językiem źródłowym a polskim
            - Tekst ma odzwierciedlać oryginał i być przystosowany do czytania na głos

        3. KOREKTA:
            - Poprawiaj i ulepszaj tłumaczenie, eliminując wszelkie błędy
            - Pomijaj znaki wodne np.: 𝑙𝘪𝑏𝓇𝑒𝑎𝒹.𝘤𝑜𝘮, libread.com
            - Sprawdź poprawność: adekwatności, antonimii, aspektualnej, dykcji, ekspresji, estetyczną, etymologicznej, fleksyjną, fonologicznej, frazeologiczną, gramatyczną, homonimii, idiomatyczności, interpunkcyjną, językową, konotacji, konwencji, kontekstową, korelacji, kulturowej, leksykalną, logiczną, metaforyczności, metryki, morfologiczną, narracji, ortoepiczną, ortograficzną, ortografii historycznej, paronimii, perspektywy, polisemii, prozodii, retoryki, rodzajową, rymu, semantyczną, składniową, słowotwórczą, stylistyczną, synonimii, syntaktyczną, tematyczną, terminologii, tonalną, transkrypcji, transliteracji, typograficzną, typu tekstu, użyteczności, wizualną, wymowy, wydźwiękową, zgodności z kontekstem, znaczenia dosłownego, znaczenia ukrytego, zrozumiałości, zwrotów

        4. DODATKOWE UWAGI I PRZYKŁADY POPRAWNOŚCI ORAZ BŁĘDÓW:
            - Poprawność Płci podejście globalne:
                ŹLE: Święty Tyris przegrał. Ona umarła. LUB Jestem pewny/pewna.
                DOBRZE: Święta Tyris przegrała. Ona umarła. LUB Na pewno.
            - Poprawność Płci podejście lokalne bez kontekstu:
                ŹLE: Zrobiłem to. LUB Zrobiłam to
                DOBRZE: To zostało zrobione przeze mnie. LUB Zostało zrobione. LUB Zrobione.
            - Idiomy:
                ŹLE: Był ich na piętach., LUB Dwa ptaki jednym kamieniem.
                DOBRZE: Deptał im po piętach. LUB Dwie pieczenie na jednym ogniu.
            - Zdania:
                ŹLE: Oczy mrugało. LUB Długi wzdychanie uciekło z jego ust. LUB Książka leżało na stole.
                DOBRZE: Oczy mrugały. LUB Długie wzdychanie uciekło z jego ust. LUB Książka leżała na stole.
                ŹLE: Nie mógł powstrzymać dreszcza.
                DOBRZE: Nie mógł powstrzymać dreszczu.
            - Zasada podmiot + orzeczenie mogą zamienić się mejscami i to nie powinno wpływać na poprawność zdania:
                ŹLE: Zaciśnięte było kawałki mięsa. Lub Zaciśnięty były kawałki mięsa.
            - Podejdź globalnie do tekstu, uwzględniając kontekst i informacje o płci postaci
            - Bądź kreatywny, dostosowując swój ton do kontekstu
            - Używaj wulgaryzmów i nie łagoć ich: kurwa, chuj, pierdolić, jebać, cholera
            - Zwróć uwagę na poprawność gramatyczną, składnię i interpunkcję
            - Bądź świadomy różnic między językiem źródłowym a polskim
            - Tekst ma odzwierciedlać oryginał i być przystosowany do czytania na głos

        3. KOREKTA:
            - Poprawiaj i ulepszaj tłumaczenie, eliminując wszelkie błędy
            - Sprawdź poprawność: adekwatności, antonimii, aspektualnej, dykcji, ekspresji, estetyczną, etymologicznej, fleksyjną, fonologicznej, frazeologiczną, gramatyczną, homonimii, idiomatyczności, interpunkcyjną, językową, konotacji, konwencji, kontekstową, korelacji, kulturowej, leksykalną, logiczną, metaforyczności, metryki, morfologiczną, narracji, ortoepiczną, ortograficzną, ortografii historycznej, paronimii, perspektywy, polisemii, prozodii, retoryki, rodzajową, rymu, semantyczną, składniową, słowotwórczą, stylistyczną, synonimii, syntaktyczną, tematyczną, terminologii, tonalną, transkrypcji, transliteracji, typograficzną, typu tekstu, użyteczności, wizualną, wymowy, wydźwiękową, zgodności z kontekstem, znaczenia dosłownego, znaczenia ukrytego, zrozumiałości, zwrotów

        4. DODATKOWE UWAGI I PRZYKŁADY POPRAWNOŚCI ORAZ BŁĘDÓW:
            - Poprawność Płci podejście globalne:
                ŹLE: Święty Tyris przegrał. Ona umarła. LUB Jestem pewny/pewna.
                DOBRZE: Święta Tyris przegrała. Ona umarła. LUB Na pewno.
            - Poprawność Płci podejście lokalne bez kontekstu:
                ŹLE: Zrobiłem to. LUB Zrobiłam to
                DOBRZE: To zostało zrobione przeze mnie. LUB Zostało zrobione. LUB Zrobione.
            - Idiomy:
                ŹLE: Był ich na piętach., LUB Dwa ptaki jednym kamieniem.
                DOBRZE: Deptał im po piętach. LUB Dwie pieczenie na jednym ogniu.
            - Zdania:
                ŹLE: Oczy mrugało. LUB Długi wzdychanie uciekło z jego ust. LUB Książka leżało na stole.
                DOBRZE: Oczy mrugały. LUB Długie wzdychanie uciekło z jego ust. LUB Książka leżała na stole.
                ŹLE: Nie mógł powstrzymać dreszcza.
                DOBRZE: Nie mógł powstrzymać dreszczu.
            - Zasada podmiot + orzeczenie mogą zamienić się mejscami i to nie powinno wpływać na poprawność zdania:
                ŹLE: Zaciśnięte było kawałki mięsa. Lub Zaciśnięty były kawałki mięsa.
                DOBRZE: Zaciśnięte były kawałki mięsa. LUB Zaciśnięty był kawałek mięsa.
                ŹLE: Mateusz była zaskoczona. LUB Zaskoczona była Mateusz. LUB Byiłem zaskoczony.
                DOBRZE: Mateusz był zaskoczony. LUB Zaskoczony był Mateusz. LUB Zaskoczyło mnie to.
            - Przekleństwa:
                ŹLE: Fuck, dick, fuck, fuck, damn
                DOBRZE: Kurwa, chuj, pierdolić, jebać, cholera
            - Błędy:
                ŹLE: Czarna Żółwia, Leciała do nich Kryli, OLD SNAKE -> Stary Węż, Męż
                DOBRZE: Czarny Żółw, Leciał do nich Kryl, -> Stary Wężu!, Mąż
            - Referencjie Do Innych Tłumaczy:
                Nie sugeruj się innym Tłumaczeniem, jest to błędne tłumaczenie które służy tylko podparciu się poprzez pewien wzór
                Nie przepisuj błędnych zwrotów, rodzaji, płci postaci, głupich, nie literackich części zdań
                BĄDŹ ROZSĄDY Np.:
                    ŹLE: ...Azure Serpents body -> ...ciała Lazurowych Węży
                    DOBRZE: ...ciała Lazurowego Węża

            - Po skończonym procesie oceń swoją prace
            - Zadanie wykonuj globalnie i krok po kroku
            - Daję Ci napiwek 1000$, jeśli wynik będzie 10/10 to otrzymasz 1000 razy tyle

        Uwzględnij dodatkowe informacjie dostępne dalej:
        """ + self.translation_notes.value + "\n\nTeraz przetłumacz poniższe napisy:\n" + text
# Obecna wersja
        #     if translated_subs is not None:
        #         translated_text: str = "".join(
        #             "◍◍{}. {}".format(
        #                 i + 1, translated_subs[i].text.replace('\n', ' ◍◍◍◍ ')
        #             )
        #             + " @@\n"
        #             for i in range((counter - 1) - len(group), counter - 1)
        #         )
        #         translated_text = translated_text.rstrip(' @@\n')
        #         prompt += "\n\nNapisy zostały wstępnie przetłumaczone przez Google Translate, NIE PRZEPISUJ TEGO GÓWNA:\n" + translated_text

        #     console.print("[green_bold]Prompt:\n", prompt, style='white_bold')

        #     response = model.generate_content(prompt)

        #     console.print("\n[green_bold]Response:\n>>>[white_bold]",
        #                   response.text.strip(), "[green_bold]<<<")

        #     translated_text: str = response.text.rstrip(" @@")
        #     if translated_text:
        #         translated_lines: List[str] = translated_text.replace(
        #             '\r\n', '\n').split(" @@\n")
        #         for i, line in enumerate(translated_lines):
        #             translated_lines[i] = line
        #         if len(translated_lines) != len(group):
        #             console.print(
        #                 f"Błąd: liczba napisów po tłumaczeniu ({len(translated_lines)}) nie jest taka sama jak przed tłumaczeniem ({len(group)})", style='red_bold')
        #         for sub, trans_text in zip(group, translated_lines):
        #             trans_text = re.sub(r"◍◍\d+\. ", "", trans_text)
        #             trans_text = trans_text.replace(" ◍◍◍◍, ", ",\n")
        #             trans_text = trans_text.replace(" ◍◍◍◍ ", "\n")
        #             trans_text = trans_text.replace(" ◍◍◍◍", "")
        #             sub.text = trans_text

        # subs.save(os.path.join(self.output_folder, filename), encoding='utf-8')
            # Obecna wersja
            if translated_subs is not None:
                translated_text: str = "".join(
                    "◍◍{}. {}".format(
                        i + 1, translated_subs[i].text.replace('\n', ' ◍◍◍◍ ')
                    )
                    + " @@\n"
                    for i in range((counter - 1) - len(group), counter - 1)
                )
                translated_text = translated_text.rstrip(' @@\n')
                prompt += "\n\nNapisy zostały wstępnie przetłumaczone przez Google Translate, NIE PRZEPISUJ TEGO GÓWNA:\n" + translated_text

            while True:
                console.print("[green_bold]Prompt:\n", prompt, style='white_bold')
                response = model.generate_content(prompt)
                console.print("\n[green_bold]Response:\n>>>[white_bold]",
                            response.text.strip(), "[green_bold]<<<")

                translated_text: str = response.text.rstrip(" @@")
                if translated_text:
                    translated_lines: List[str] = translated_text.replace(
                        '\r\n', '\n').split(" @@\n")
                    for i, line in enumerate(translated_lines):
                        translated_lines[i] = line
                    if len(translated_lines) != len(group):
                        console.print(
                            f"BŁĄD: liczba napisów po tłumaczeniu ({len(translated_lines)}) nie jest taka sama jak przed tłumaczeniem ({len(group)})", style='red_bold')
                        prompt = f"\n\nUWAGA: POPRZEDNIO JAK WYSŁAŁEM DO CIEBIE TEN PROMPT TO ZWRÓCIŁEŚ PODAŁEŚ NIEPRAWIDŁOWĄ LICZBĘ NAPISÓW, POPRAW TO I WYŚLIJ PONOWNIE np.: - Jak otrzymasz 50 napisów, zwróć 50 napisów ze znakami ◍◍ na początku i @ w odpowidnich miejscach. | BŁĄD: liczba napisów po tłumaczeniu ({len(translated_lines)}) nie jest taka sama jak przed tłumaczeniem ({len(group)})\n\n" + prompt
                        continue
                    else:
                        for sub, trans_text in zip(group, translated_lines):
                            trans_text = re.sub(r"◍◍\d+\. ", "", trans_text)
                            trans_text = trans_text.replace(" ◍◍◍◍, ", ",\n")
                            trans_text = trans_text.replace(" ◍◍◍◍ ", "\n")
                            trans_text = trans_text.replace(" ◍◍◍◍", "")
                            sub.text = trans_text
                        break

            subs.save(os.path.join(self.output_folder,
                    filename), encoding='utf-8')

    def translate_srt(self, filename: str, translated_line_count: int):
        """
            Translates subtitles using the selected translation mode.

            Attributes:
                - filename (str): The name of the subtitle file.
                - translated_line_count (int): The number of lines to translate at a time.

            Depending on the selected translation mode, this method calls either the translate_gemini or translate_google_gemini method to translate the subtitles. If an unknown translation mode is selected, it prints an error message.
        """
        if self.translation_mode.value == 'Gemini':
            self.translate_gemini(filename, translated_line_count)
        elif self.translation_mode.value == 'Gemini + Google Translator':
            self.translate_google_gemini(filename, translated_line_count)
        else:
            console.print("Nieznany tryb tłumaczenia.",
                          style='red_bold')

    # Metody zarządzające interfejsem użytkownika
    def clear_folder(self, folder_name):
        """
            Clears all files from a specified folder.

            Attributes:
                - folder_name (str): The name of the folder to be cleared.

            This method checks if the folder exists, and if it does, it iterates over all files in the folder and deletes them. If the folder does not exist, it prints an error message.
        """
        if os.path.exists(folder_name):
            for filename in os.listdir(folder_name):
                file_path = os.path.join(folder_name, filename)
                try:
                    if os.path.isfile(file_path) or os.path.islink(file_path):
                        os.unlink(file_path)
                    elif os.path.isdir(file_path):
                        shutil.rmtree(file_path)
                except Exception as e:
                    console.print(f"Nie udało się usunąć {file_path}. Powód: {e}.",
                                  style='red_bold')
            console.print(f"Usunięto wszystkie pliki z folderu {folder_name}.",
                          style='green_bold')
        else:
            console.print(
                f"Folder {folder_name} nie istnieje.", style='red_bold')

    def clear_input_folder(self, b):
        """
            Clears all files from the input folder.

            Attributes:
                - b: Button widget instance. It's not used in the function, but is required for the button's on_click event.
        """
        self.clear_folder(self.input_folder)

    def clear_output_folder(self, b):
        """
            Clears all files from the output folder.

            Attributes:
                - b: Button widget instance. It's not used in the function, but is required for the button's on_click event.
        """
        self.clear_folder(self.output_folder)

    def upload_srt_files(self, b=None):
        """
            Uploads .srt files.

            Attributes:
                - b (optional): Button widget instance. It's not used in the function, but is required for the button's on_click event.
        """
        with suppress(Exception):
            uploaded = files.upload()

            for fn in uploaded.keys():
                console.print('User uploaded file "{name}" with length {length} bytes'.format(
                    name=fn, length=len(uploaded[fn])), style='green_bold')

            for fn in uploaded.keys():
                shutil.move(fn, f'{self.input_folder}/{fn}')

    def update_auto_download_status(self, change):
        """
            Updates the status of the auto download feature.

            Attributes:
                - change: Contains the information about the change.
        """
        self.auto_download = change['new']

    def download_files(self, b=None):
        """
            Downloads all .srt files from the output folder.

            Attributes:
                - b (optional): Button widget instance. It's not used in the function, but is required for the button's on_click event.
        """
        zip_filename = 'translated_files.zip'
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            for file in os.listdir(self.output_folder):
                if file.endswith('.srt'):
                    zipf.write(os.path.join(self.output_folder, file))
        files.download(zip_filename)

    def display(self):
        """
            Displays the user interface for the Gemini Translator.
        """
        display(HTML('<h1>Gemini Translator - Tłumacz Napisów</h1>'))
        display(HTML('<h2>USTAWIENIA</h2>'))
        display(self.api_key, self.temperature, self.top_p, self.top_k, self.max_output_tokens, self.translated_line_count,
                self.safety_harassment, self.safety_hate_speech, self.safety_sexually_explicit, self.safety_dangerous_content, self.translation_notes)
        display(HTML('<h2>USUŃ:</h2>'))
        display(self.clear_input_button, self.clear_output_button)
        display(HTML('<h2>TŁUMACZ:</h2>'))
        display(self.translation_mode, self.upload_button,
                self.auto_download_button, self.start_translate_button)

    @execution_timer
    def start_translation(self, b):
        """
            Starts the translation process.

            Attributes:
                - b: Button widget instance. It's not used in the function, but is required for the button's on_click event.
        """
        console.print("Rozpoczynam tłumaczenie...", style='green_bold')
        files = [filename for filename in os.listdir(
            self.input_folder) if filename.endswith('.srt')]
        sorted_files = natsorted(files)
        for filename in sorted_files:
            self.translate_srt(filename, self.translated_line_count.value)
        # Po zakończeniu tłumaczenia, wyświetlamy przycisk pobierania
        display(self.download_button)
        if self.auto_download:
            self.download_files()


def main():
    # Zmieana katalogu roboczego i utworzenie folderów

    # Google Colab
    # os.chdir('/content/')
    # os.makedirs('GeminiTranslator', exist_ok=True)

    # os.chdir('/content/GeminiTranslator/')
    # os.makedirs('input', exist_ok=True)
    # os.makedirs('output', exist_ok=True)

    # Google Drive Shared Drive Dysk_0
    os.chdir('/content/drive/Shareddrives/Dysk_0/')
    os.makedirs('GeminiTranslator', exist_ok=True)

    os.chdir('/content/drive/Shareddrives/Dysk_0/GeminiTranslator/')
    os.makedirs('input', exist_ok=True)
    os.makedirs('output', exist_ok=True)

    # Utwórz instancję klasy GeminiTranslator
    translator = GeminiTranslator(input_folder='input', output_folder='output',
                                  api_key='API_KEY')

    translator.display()


if __name__ == '__main__':
    main()


# 2. Gemini Podstawa Dla Modelu: gemini-pro ✅


In [ ]:
!pip install -U -q google-generativeai
import google.generativeai as genai

genai.configure(api_key='API_KEY')

# Set up the model
generation_config = {
    "temperature": 0.9,
    "top_p": 1,
    "top_k": 1,
    "max_output_tokens": 1000000,
}

safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_ONLY_HIGH"
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_ONLY_HIGH"
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_ONLY_HIGH"
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_ONLY_HIGH"
    },
]

model = genai.GenerativeModel(model_name="gemini-pro",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

prompt_parts = [
    """
PROMPT *_*
    """
]

response = model.generate_content(prompt_parts)
print(response.text)

In [ ]:
import requests

def get_location():
    info = requests.get('https://ipinfo.io/')
    data = info.json()
    print(data)

get_location()

# INNE

##### Copyright 2023 Google LLC

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

### Install & import

In [ ]:
!pip install -U -q google-generativeai

In [ ]:
# Install the client library and import necessary modules.
import google.generativeai as genai

import base64
import io
import json
import mimetypes
import pathlib
import pprint
import requests

import PIL.Image
import IPython.display
from IPython.display import Markdown



### Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

## Set the API key

Add your API_KEY to the secrets manager in the left pannel "🔑".

In [ ]:
from google.colab import userdata

API_KEY=userdata.get('API_KEY')

In [ ]:
# Configure the client library by providing your API key.
genai.configure(api_key=API_KEY)

### Parse the arguments

In [ ]:
model = 'gemini-pro' # @param {isTemplate: true}
contents_b64 = 'W3sicGFydHMiOltdfV0=' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MC45LCJ0b3BfcCI6MSwidG9wX2siOjEsIm1heF9vdXRwdXRfdG9rZW5zIjoxMDAwMDAwLCJzdG9wX3NlcXVlbmNlcyI6W119' # @param {isTemplate: true}
safety_settings_b64 = 'W3siY2F0ZWdvcnkiOiJIQVJNX0NBVEVHT1JZX0hBUkFTU01FTlQiLCJ0aHJlc2hvbGQiOiJCTE9DS19PTkxZX0hJR0gifSx7ImNhdGVnb3J5IjoiSEFSTV9DQVRFR09SWV9IQVRFX1NQRUVDSCIsInRocmVzaG9sZCI6IkJMT0NLX09OTFlfSElHSCJ9LHsiY2F0ZWdvcnkiOiJIQVJNX0NBVEVHT1JZX1NFWFVBTExZX0VYUExJQ0lUIiwidGhyZXNob2xkIjoiQkxPQ0tfT05MWV9ISUdIIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfREFOR0VST1VTX0NPTlRFTlQiLCJ0aHJlc2hvbGQiOiJCTE9DS19PTkxZX0hJR0gifV0=' # @param {isTemplate: true}

contents = json.loads(base64.b64decode(contents_b64))
generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

In [ ]:
contents

In [ ]:
generation_config

In [ ]:
safety_settings

### Load image data from Drive-IDs

In [ ]:
for content in contents:
  for n, part in enumerate(content['parts']):
    if image:=part.get('image', None):
      if drive_id:=image.get('drive_id', None):
        path = next(pathlib.Path(f'/gdrive/.shortcut-targets-by-id/{drive_id}').glob('*'))
        data = path.read_bytes()
        mime_type, _ = mimetypes.guess_type(path)
      elif image_url:=image.get('image_url', None):
        response = requests.get(image_url)
        data = response.content
        mime_type = response.headers['content-type']
      else:
        raise ValueError('Either drive_id or image_url must be provided.')

      if mime_type is None:
        # Guess!
        mime_type = 'image/png'

      blob = {'data': data, 'mime_type': mime_type}
      content['parts'][n] = {'inline_data': blob}

In [ ]:
import IPython.display
import PIL.Image
import io

for content in contents:
    for part in content['parts']:
        if text := part.get('text', None):
            print(text)
        elif data := part.get('inline_data', None):
            img = PIL.Image.open(io.BytesIO(data['data']))
            img.thumbnail([512,512])
            IPython.display.display(img)
    print('_'*80)


### Call the API

In [ ]:
# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=False)

In [ ]:
if generation_config.get('candidate_count', 1) == 1:
  display(Markdown(response.text))
else:
  print(response.candidates)

In [ ]:
response.candidates

In [ ]:
response.prompt_feedback